In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [2]:
files_list = ['2024-10-15_Invest_nv_SUEK.xlsx', '2024-10-15_Invest_nv_EUROCHEM.xlsx']
old_data_SU = pd.read_excel(Output_path+files_list[0])
old_data_ECH = pd.read_excel(Output_path+files_list[1])
old_data = pd.concat([old_data_SU,old_data_ECH]).reset_index(drop=True)

In [3]:
new_data = export_from_RISKCUSTOM(f'''select * 
from RISKACCESS."unifiedOcp"
where "sourceTable" = 'sapBwInvestmentReport'
and "reportDate" = TO_DATE('30.09.24', 'DD.MM.YY')
--fetch first 10 rows only''')
new_data.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(37665, 40)

In [4]:
data_OCP_work = new_data[['holding','buCode','salesUnitsId','ocpSegment','counterpartySegment','counterpartyCode','effectiveCurrency','volumeEffectiveCurrency','maturityDateTo']]
data_OCP_work.maturityDateTo = data_OCP_work.maturityDateTo.astype(str).apply(lambda x: '-'.join([x[:-3],'01']))
data_OCP_work.maturityDateTo = pd.to_datetime(data_OCP_work.maturityDateTo)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_4132\2119538208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.maturityDateTo = data_OCP_work.maturityDateTo.astype(str).apply(lambda x: '-'.join([x[:-3],'01']))
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_4132\2119538208.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.maturityDateTo = pd.to_datetime(data_OCP_work.maturityDateTo)


In [5]:
# merge sum to dates
data_OCP_work.counterpartyCode = data_OCP_work.counterpartyCode.astype(str)
old_data.Контрагент = old_data.Контрагент.astype(str)
data_OCP_work.ocpSegment = data_OCP_work.ocpSegment.fillna('External')

leaft_list = ['holding','Comp_segment','Comp_code','Контрагент','currency','КалендГод/Месяц']
ver_cols = ['Версия_035', 'Версия_045','Версия_075', 'Версия_085']
right_list = ['holding','ocpSegment','buCode','counterpartyCode','effectiveCurrency','maturityDateTo']

old_data['Amount_abs'] = old_data['Amount'].abs()
old_data_pivot = old_data.groupby(as_index=False,dropna=False, by=leaft_list)[['Amount_abs']+ver_cols].agg(sum)
data_OCP_work['Amount_abs'] = data_OCP_work['volumeEffectiveCurrency'].abs()
data_OCP_work_pivot = data_OCP_work.groupby(as_index=False,dropna=False, by=right_list)['Amount_abs'].agg(sum)
# merge
merge_data = old_data_pivot.merge(data_OCP_work_pivot, how='outer', left_on=leaft_list, right_on=right_list, suffixes=['_old','_OCP'])
# merge_data[merge_data.Amount_abs_old!=merge_data.Amount_abs_OCP]

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_4132\3146694621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.counterpartyCode = data_OCP_work.counterpartyCode.astype(str)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_4132\3146694621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.ocpSegment = data_OCP_work.ocpSegment.fillna('External')
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_4132\3146694621.py:11: FutureWarning: The provided callable <built-in functi

In [16]:
old_data.shape[0], new_data.shape[0]

(35743, 37665)

In [32]:
old_data[old_data['Amount']!=0].shape[0], new_data[~new_data['volumeEffectiveCurrency'].isna()].shape[0]

(17706, 18767)

In [6]:
merge_data = is_approximately_equal_for_cols(merge_data,'Amount_abs_old','Amount_abs_OCP')

In [7]:
merge_data[merge_data['Diff__more_0.4']==True]

,holding,Comp_segment,Comp_code,Контрагент,currency,КалендГод/Месяц,Amount_abs_old,Версия_035,Версия_045,Версия_075,Версия_085,ocpSegment,buCode,counterpartyCode,effectiveCurrency,maturityDateTo,Amount_abs_OCP,Diff__more_0.4
427,EUROCHEM,RUFERT,E100,1000056464,CNY,2024-10-01,1605000.14,3250000.14,1926000.17,1605000.14,3250000.14,RUFERT,E100,1000056464,CNY,2024-10-01,160500.0140,True
586,EUROCHEM,RUFERT,E100,1000138907,CNY,2024-12-01,82722.58,112242.58,41241062.92,34367552.39,112242.58,RUFERT,E100,1000138907,CNY,2024-12-01,8272.2580,True
587,EUROCHEM,RUFERT,E100,1000138907,CNY,2025-01-01,82722.58,112242.58,99267.18,82722.58,112242.58,RUFERT,E100,1000138907,CNY,2025-01-01,8272.2580,True
588,EUROCHEM,RUFERT,E100,1000138907,CNY,2025-02-01,82722.58,112242.58,99267.18,82722.58,112242.58,RUFERT,E100,1000138907,CNY,2025-02-01,8272.2580,True
589,EUROCHEM,RUFERT,E100,1000138907,CNY,2025-03-01,82722.58,112242.58,99267.18,82722.58,112242.58,RUFERT,E100,1000138907,CNY,2025-03-01,8272.2580,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11066,SUEK,SUEK RU,6700,1000066814,CNY,2024-11-01,4254509.00,4254509.00,4339599.18,3616332.65,4254509.00,SUEK RU,6700,1000066814,CNY,2024-11-01,425450.9000,True
11089,SUEK,SUEK RU,6700,1000071921,CNY,2024-12-01,3517440.00,3517440.00,3587788.80,2989824.00,3517440.00,SUEK RU,6700,1000071921,CNY,2024-12-01,351744.0000,True
11154,SUEK,SUEK RU,6700,1000096074,CNY,2024-10-01,1436163.58,1488742.34,1450105.45,1208421.21,1488742.34,SUEK RU,6700,1000096074,CNY,2024-10-01,143616.3580,True
11194,SUEK,SUEK RU,6700,1000138907,CNY,2024-11-01,240000.02,3548778.72,240000.02,200000.02,3548778.72,SUEK RU,6700,1000138907,CNY,2024-11-01,25200.0021,True


In [8]:
# to excel
Output_file = f'{str(date.today())}_ivest_vs_OCP.xlsx'
with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
    merge_data[merge_data['Diff__more_0.4']==True].to_excel(writer, index=False, sheet_name='compare')
    old_data.to_excel(writer, index=False, sheet_name='data_script')
    new_data.to_excel(writer, index=False, sheet_name='data_unOCP')
